In [ ]:
import glob
from pathlib import Path
import pandas as pd

runs = pd.DataFrame({'run': glob.glob('smac3-output*/run*')}).sort_values('run').reset_index(drop=True)
runs

In [ ]:
runs['start'] = pd.to_datetime(
    runs.run\
        .str.partition('/')[0]\
        .str.removeprefix('smac3-output_')\
        .str.rpartition('_')[0]\
        .str.replace('_', 'T'),
    format='ISO8601'
)
runs.run = runs.run.map(Path)
runs

In [ ]:
import json
stats = pd.DataFrame(json.load((r / 'stats.json').open()) for r in runs.run)
stats

In [ ]:
df = runs.join(stats)

In [ ]:
import shutil
def remove_unfinished_runs(df):
    old = df.iloc[:-1]  # the latest run can still be in progress, exclude it
    for r in old[old.submitted_ta_runs != old.finished_ta_runs].run:
        shutil.rmtree(r)

In [ ]:
df = df[df.finished_ta_runs == 10_000].set_index('run')
df

In [ ]:
import json
best = pd.DataFrame(
    pd.read_json((r / 'traj.json'), lines=True).iloc[-1]
    for r in df.index)
best.index=df.index
result = df.join(best).set_index('start').drop(columns=['submitted_ta_runs', 'finished_ta_runs'])
result


In [ ]:
for incumbent in result.incumbent:
    print(incumbent)